In [ ]:
import os
from urllib.request import urlretrieve
import pandas as pd
from datasets import load_dataset

table_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/metadata-large.parquet'

pbar = None

if not os.path.exists('metadata.parquet'):
    print("retrieving metadata file")
    urlretrieve(table_url, 'metadata.parquet')
# Read the table using Pandas\n",
metadata_df = pd.read_parquet('metadata.parquet')

In [ ]:
! pip install https://huggingface.co/spacy/en_core_web_trf/resolve/main/en_core_web_trf-any-py3-none-any.whl

In [ ]:
import en_core_web_trf
nlp = en_core_web_trf.load()


In [ ]:
metadata_df["prompt"].head(10)

In [ ]:

first_n_unique_prompts = metadata_df["prompt"].sample(n=1000, random_state=42).drop_duplicates().head(100)
display(first_n_unique_prompts.head(5).tolist())

In [ ]:
from datasets import Dataset, Features
from spacy import displacy
from spacy.symbols import nsubj, VERB


def process(prompts):
    out = []
    for prompt in prompts:
        doc = nlp(prompt)
        # displacy.render(doc, style="dep")
        valid_tokens = []
        prompt
        # find the first chunk with either an entity or a proper noun.
        for chunk in doc.noun_chunks:
            proper_nouns = [token for token in chunk if token.pos_ == "PROPN"]
            proper_ents, non_proper_ents = [], []
            for ent in chunk.ents:
                if ent.label_ == "PERSON" or ent.label_ == "ORG":
                    proper_ents.append(ent)
                else:
                    non_proper_ents.append(ent)
            valid_tokens.append(chunk.root.text)
            if len(non_proper_ents) > 0:
                valid_tokens.append(chunk.text)
                break
            elif len(proper_nouns) > 0 and len(proper_ents) == 0:
                valid_tokens.append(chunk.text)
                break

        # print("token deps")
        valid_tokens = [tok for i, tok in enumerate(valid_tokens) if tok not in valid_tokens[:i]]
        out.append(" ".join(valid_tokens))
    return out


# display([(p, process(p)) for p in [
#     "stunning goddess of beers portrait, clear eyes and dark skin. realistic, symmetrical face. art by bowater charlie, mark brooks, julie bell, arian mark, tony sandoval "
# ]])
display([(p, process(p)) for p in first_n_unique_prompts[:100]])

In [53]:
prompt_only_df = first_n_unique_prompts.to_frame()
dataset = Dataset.from_pandas(prompt_only_df)
dataset = dataset.map(process, batched=True, batch_size=1000, num_proc=16)


Map (num_proc=16):   0%|          | 0/100 [00:00<?, ? examples/s]

ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'datasets.formatting.formatting.LazyBatch'>